# Aula 9.Ex - Malhas e Texturas - Mapeamento de Texturas

## Matheus Bermudes Viana
## 11849797
## Ex de Presença

### Primeiro, importamos as bibliotecas necessárias.
Verifique no código anterior um script para instalar as dependências necessárias (OpenGL e GLFW) antes de prosseguir.

In [825]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

### Inicializando janela

In [826]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1600
largura = 1200
window = glfw.create_window(largura, altura, "Malhas e Texturas", None, None)
glfw.make_context_current(window)

### GLSL (OpenGL Shading Language)

Aqui veremos nosso primeiro código GLSL.

É uma linguagem de shading de alto nível baseada na linguagem de programação C.

Estamos escrevendo código GLSL como se "strings" de uma variável (mas podemos ler de arquivos texto). Esse código, depois, terá que ser compilado e linkado ao nosso programa. 

Aprenderemos GLSL conforme a necessidade do curso. Usaremos uma versão do GLSL mais antiga, compatível com muitos dispositivos.

### GLSL para Vertex Shader

No Pipeline programável, podemos interagir com Vertex Shaders.

No código abaixo, estamos fazendo o seguinte:

* Definindo uma variável chamada position do tipo vec3.
* Definindo matrizes Model, View e Projection que acumulam transformações geométricas 3D e permitem navegação no cenário.
* void main() é o ponto de entrada do nosso programa (função principal).
* gl_Position é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a posição de um vértice. Observe que todo vértice tem 4 coordenadas, por isso combinamos nossa variável vec2 com uma variável vec4. Além disso, modificamos nosso vetor com base nas transformações Model, View e Projection.

In [827]:
vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;

        varying vec2 out_texture;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_normal = vec3( model *vec4(normals, 1.0));            

        }
        """

### GLSL para Fragment Shader

No Pipeline programável, podemos interagir com Fragment Shaders.

No código abaixo, estamos fazendo o seguinte:

* void main() é o ponto de entrada do nosso programa (função principal).
* gl_FragColor é uma variável especial do GLSL. Variáveis que começam com 'gl_' são desse tipo. Nesse caso, determina a cor de um fragmento. Nesse caso é um ponto, mas poderia ser outro objeto (ponto, linha, triangulos, etc).

### Possibilitando modificar a cor.

Nos exemplos anteriores, a variável gl_FragColor estava definida de forma fixa (com cor R=0, G=0, B=0).

Agora, criaremos uma variável do tipo "uniform", de quatro posições (vec4), para receber o dado de cor do nosso programa rodando em CPU.

In [828]:
fragment_code = """
        uniform vec4 color;
        varying vec2 out_texture;
        uniform sampler2D samplerTexture;
        
        void main(){
            vec4 texture = texture2D(samplerTexture, out_texture);
            gl_FragColor = texture;
        }
        """

### Requisitando slot para a GPU para nossos programas Vertex e Fragment Shaders

In [829]:
# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

### Associando nosso código-fonte aos slots solicitados

In [830]:
# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

### Compilando o Vertex Shader

Se há algum erro em nosso programa Vertex Shader, nosso app para por aqui.

In [831]:
# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

### Compilando o Fragment Shader

Se há algum erro em nosso programa Fragment Shader, nosso app para por aqui.

In [832]:
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

### Associando os programas compilado ao programa principal

In [833]:
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

### Linkagem do programa

In [834]:
# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Preparando dados para enviar a GPU

Nesse momento, compilamos nossos Vertex e Program Shaders para que a GPU possa processá-los.

Por outro lado, as informações de vértices geralmente estão na CPU e devem ser transmitidas para a GPU.

### Carregando Modelos (vértices e texturas) a partir de Arquivos

A função abaixo carrega modelos a partir de arquivos no formato WaveFront.


Para saber mais sobre o modelo, acesse: https://en.wikipedia.org/wiki/Wavefront_.obj_file


Nos slides e vídeo da Aula, descrevemos o funcionamento desse formato.

In [835]:
def load_model_from_file(filename):
    """Loads a Wavefront OBJ file. """
    objects = {}
    vertices = []
    texture_coords = []
    faces = []

    material = None

    # abre o arquivo obj para leitura
    for line in open(filename, "r"): ## para cada linha do arquivo .obj
        if line.startswith('#'): continue ## ignora comentarios
        values = line.split() # quebra a linha por espaço
        if not values: continue


        ### recuperando vertices
        if values[0] == 'v':
            vertices.append(values[1:4])


        ### recuperando coordenadas de textura
        elif values[0] == 'vt':
            texture_coords.append(values[1:3])

        ### recuperando faces 
        elif values[0] in ('usemtl', 'usemat'):
            material = values[1]
        elif values[0] == 'f':
            face = []
            face_texture = []
            for v in values[1:]:
                w = v.split('/')
                face.append(int(w[0]))
                if len(w) >= 2 and len(w[1]) > 0:
                    face_texture.append(int(w[1]))
                else:
                    face_texture.append(0)

            faces.append((face, face_texture, material))

    model = {}
    model['vertices'] = vertices
    model['texture'] = texture_coords
    model['faces'] = faces

    return model

In [836]:
glEnable(GL_TEXTURE_2D)
qtd_texturas = 10
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    print(texture_id)
    print(img_textura)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
    #image_data = np.array(list(img.getdata()), np.uint8)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### A lista abaixo armazena todos os vertices carregados dos arquivos

In [837]:
vertices_list = []    
textures_coord_list = []

indice_vertice_objeto = [0,0]
print(indice_vertice_objeto[1])

0


In [838]:
# Função para visualizar melhor quais vértices pertencem a cada objeto

def print_vertices(modelo, len_vertices_list):
    global indice_vertice_objeto
    elemento_atual = modelo['faces'][0][2]
    contagem_vertices_elemento = [-3,0,0,0,0,0,0,0,0]
    i = 0

    # Para cada face do modelo
    for face in modelo['faces']:
        # Se o elemento atual for igual ao elemento da face atual
        # então incrementa a contagem de vértices do elemento atual
        contagem_vertices_elemento[i] += 3
        proximo_elemento = face[2]
        # Se o elemento atual for diferente do elemento da face atual
        # então imprime a contagem de vértices do elemento atual
        # e atualiza o elemento atual
        if(proximo_elemento != elemento_atual):
            indice_vertice_objeto[0] = indice_vertice_objeto[1]
            indice_vertice_objeto[1] += contagem_vertices_elemento[i]
            print("Parte:", elemento_atual, "\tVértices:", contagem_vertices_elemento[i], "\tInício:", indice_vertice_objeto[0], "\tFim:", indice_vertice_objeto[1], "\n")
            elemento_atual = proximo_elemento
            i += 1

    # Imprime a contagem de vértices do último elemento 
    indice_vertice_objeto[0] = indice_vertice_objeto[1]
    indice_vertice_objeto[1] += contagem_vertices_elemento[i]+3
    print("Parte:", proximo_elemento, "\tVértices:", contagem_vertices_elemento[i]+3, "\tInício:", indice_vertice_objeto[0], "\tFim:", indice_vertice_objeto[1], "\n")

    # Atualiza a contagem de vértices
    indice_vertice_objeto[0] = len_vertices_list
    indice_vertice_objeto[1] = len_vertices_list
    print(indice_vertice_objeto)

### Carregamos cada modelo e definimos funções para desenhá-los

In [839]:
modelo = load_model_from_file('caixa/caixa.obj')
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo cube.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo cube.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(0,'caixa/caixa2.jpg')

print_vertices(modelo, len(vertices_list))


Processando modelo cube.obj. Vertice inicial: 0
Processando modelo cube.obj. Vertice final: 36
0
caixa/caixa2.jpg
Parte: Material.002 	Vértices: 36 	Início: 0 	Fim: 36 

[36, 36]


In [840]:
modelo = load_model_from_file('terreno/terreno2.obj')
global indice_vertice_objeto
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo terreno.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo terreno.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(1,'terreno/rock.jpg')

print_vertices(modelo, len(vertices_list))


Processando modelo terreno.obj. Vertice inicial: 36
Processando modelo terreno.obj. Vertice final: 42
1
terreno/rock.jpg
Parte: None 	Vértices: 6 	Início: 36 	Fim: 42 

[42, 42]


In [841]:
# modelo = load_model_from_file('casa/casa.obj')
modelo = load_model_from_file('casa/House/casanova.obj')
# print(modelo)
### inserindo vertices do modelo no vetor de vertices
print('Processando modelo casa.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    # for texture_id in face[1]:
        # textures_coord_list.append( modelo['texture'][texture_id-1] )
print(len(vertices_list))
print('Processando modelo casa.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(2,'casa/House/texture.png')
load_texture_from_file(3,'casa/House/texturedarker.png')
load_texture_from_file(4,'casa/House/red.png')
print(len(vertices_list))

print_vertices(modelo, len(vertices_list))



Processando modelo casa.obj. Vertice inicial: 42
5388
Processando modelo casa.obj. Vertice final: 5388
2
casa/House/texture.png
3
casa/House/texturedarker.png
4
casa/House/red.png
5388
Parte: Edges 	Vértices: 3084 	Início: 42 	Fim: 3126 

Parte: Walls 	Vértices: 993 	Início: 3126 	Fim: 4119 

Parte: Ceilings 	Vértices: 39 	Início: 4119 	Fim: 4158 

Parte: Roof 	Vértices: 222 	Início: 4158 	Fim: 4380 

Parte: Glass 	Vértices: 36 	Início: 4380 	Fim: 4416 

Parte: Sliding_Glass_Door 	Vértices: 168 	Início: 4416 	Fim: 4584 

Parte: Bar 	Vértices: 192 	Início: 4584 	Fim: 4776 

Parte: Floor 	Vértices: 228 	Início: 4776 	Fim: 5004 

Parte: Window_Frames 	Vértices: 384 	Início: 5004 	Fim: 5388 

[5388, 5388]


In [842]:
modelo = load_model_from_file('monstro/monstro.obj')

### inserindo vertices do modelo no vetor de vertices
print('Processando modelo monstro.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo monstro.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(5,'monstro/monstro2.jpg')

print_vertices(modelo, len(vertices_list))


Processando modelo monstro.obj. Vertice inicial: 5388
Processando modelo monstro.obj. Vertice final: 11496
5
monstro/monstro2.jpg
Parte: None 	Vértices: 6108 	Início: 5388 	Fim: 11496 

[11496, 11496]


In [843]:
# modelo = load_model_from_file('notebook/untitled.obj')
modelo = load_model_from_file('notebook/Lowpoly_Notebook_2.obj')


### inserindo vertices do modelo no vetor de vertices
print('Processando modelo notebook.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    for texture_id in face[1]:
        textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo notebook.obj. Vertice final:',len(vertices_list))
### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(6,'notebook/textures/Lowpoly_Laptop_1.jpg')
load_texture_from_file(7,'notebook/textures/Lowpoly_Laptop_2.jpg')

print_vertices(modelo, len(vertices_list))


Processando modelo notebook.obj. Vertice inicial: 11496
Processando modelo notebook.obj. Vertice final: 12095
6
notebook/textures/Lowpoly_Laptop_1.jpg
7
notebook/textures/Lowpoly_Laptop_2.jpg
Parte: Lowpoly_Body 	Vértices: 201 	Início: 11496 	Fim: 11697 

Parte: Lowpoly_Screen 	Vértices: 282 	Início: 11697 	Fim: 11979 

[12095, 12095]


In [844]:
# modelo = load_model_from_file('monstro/carro.obj')
modelo = load_model_from_file('carro/Car-Model/carrotriangle.obj')

# modelo = load_model_from_file('caixa/caixa.obj')


### inserindo vertices do modelo no vetor de vertices
print('Processando modelo carrotriangle.obj. Vertice inicial:',len(vertices_list))
for face in modelo['faces']:
    for vertice_id in face[0]:
        vertices_list.append( modelo['vertices'][vertice_id-1] )
    # for texture_id in face[1]:
    #     textures_coord_list.append( modelo['texture'][texture_id-1] )
print('Processando modelo carrotriangle.obj. Vertice final:',len(vertices_list))

### inserindo coordenadas de textura do modelo no vetor de texturas


### carregando textura equivalente e definindo um id (buffer): use um id por textura!
load_texture_from_file(8,'carro/Car-Model/vermelho.png')
load_texture_from_file(9,'carro/Car-Model/azul.png')
load_texture_from_file(10,'carro/Car-Model/vinho.png')
load_texture_from_file(11,'carro/Car-Model/cinzaclaro.png')
load_texture_from_file(12,'carro/Car-Model/preto.png')
load_texture_from_file(13,'carro/Car-Model/laranja.png')


# load_texture_from_file(4,'notebook/textures/moodle.jpg')
# load_texture_from_file(5,'notebook/textures/Lowpoly_Laptop_2.jpg')


# load_texture_from_file(4,'caixa/caixa2.jpg')

print_vertices(modelo, len(vertices_list))



Processando modelo carrotriangle.obj. Vertice inicial: 12095
Processando modelo carrotriangle.obj. Vertice final: 19787
8
carro/Car-Model/vermelho.png
9
carro/Car-Model/azul.png
10
carro/Car-Model/vinho.png
11
carro/Car-Model/cinzaclaro.png
12
carro/Car-Model/preto.png
13
carro/Car-Model/laranja.png
Parte: Body 	Vértices: 606 	Início: 12095 	Fim: 12701 

Parte: Black 	Vértices: 168 	Início: 12701 	Fim: 12869 

Parte: Window 	Vértices: 186 	Início: 12869 	Fim: 13055 

Parte: Bumpers 	Vértices: 480 	Início: 13055 	Fim: 13535 

Parte: Lights 	Vértices: 12 	Início: 13535 	Fim: 13547 

Parte: Bottom 	Vértices: 144 	Início: 13547 	Fim: 13691 

Parte: Tires 	Vértices: 5376 	Início: 13691 	Fim: 19067 

Parte: Wheels 	Vértices: 720 	Início: 19067 	Fim: 19787 

[19787, 19787]


### Para enviar nossos dados da CPU para a GPU, precisamos requisitar slots.

Agora requisitamos dois slots.
* Um para enviar coordenadas dos vértices.
* Outro para enviar coordenadas de texturas.

In [845]:
# Request a buffer slot from GPU
buffer = glGenBuffers(2)

###  Enviando coordenadas de vértices para a GPU

In [846]:
vertices = np.zeros(len(vertices_list), [("position", np.float32, 3)])
vertices['position'] = vertices_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)

###  Enviando coordenadas de textura para a GPU

In [847]:
textures = np.zeros(len(textures_coord_list), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = textures_coord_list


# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)

### Desenhando nossos modelos
* Cada modelo tem um Model para posicioná-los no mundo.
* É necessário saber qual a posição inicial e total de vértices de cada modelo.
* É necessário indicar qual o ID da textura do modelo.

In [848]:
def desenha_caixa():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = 0.0; t_z = 15.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 0)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 0, 36) ## renderizando

In [849]:
def desenha_terreno():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.01; t_z = 0.0;
    
    # escala
    s_x = 20.0; s_y = 20.0; s_z = 20.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 1)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 36, 42-36) ## renderizando

In [850]:
def desenha_casa():
    prev = 42

    edges = 3084 + prev
    walls = 993 + prev 
    ceilings = 39 + prev 
    roof = 222 + prev 
    resto = 5388-roof
    # aplica a matriz model
    
    # rotacao
    angle = 0.0;
    r_x = 0.0; r_y = 0.0; r_z = 1.0;
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0;
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0;
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)


    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 2)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 42, 3084 	 ) ## renderizando
    
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 3)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 3126, 993 	 ) ## renderizando

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 4)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 4119 , 39 	 ) ## renderizando

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 4)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 4158 , 222 	 ) ## renderizando

    glDrawArrays(GL_TRIANGLES, 4380 , 36 	) ## renderizando





    # #define id da textura do modelo
    # glBindTexture(GL_TEXTURE_2D, 2)
    # # desenha o modelo
    # glDrawArrays(GL_TRIANGLES, 42, 5388-42) ## renderizando

In [851]:
def desenha_monstro(rotacao_inc):
    
    
    # aplica a matriz model
    
    # rotacao
    angle = rotacao_inc;
    r_x = 0.0; r_y = 1.0; r_z = 0.0
    
    # translacao
    t_x = 0.0; t_y = -1.0; t_z = 0.0
    
    # escala
    s_x = 1.0; s_y = 1.0; s_z = 1.0
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 3)
    
    
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 5388, 11496-5388) ## renderizando

In [852]:
def desenha_note():
    
    
    # aplica a matriz model
    
    # rotacao
    angle = 0;
    r_x = 1.0; r_y = 1.0; r_z = 1.0
    
    # translacao
    t_x = 0; t_y = .2; t_z = 0.0
    
    # escala
    s_x = .1; s_y = .1; s_z = .1
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)
       
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 6)
    # desenha o modelo
    
    glDrawArrays(GL_TRIANGLES , 11496, 348) ## renderizando

    # #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 7)
    # # desenha o modelo
    glDrawArrays(GL_TRIANGLES, 11844, 483) ## renderizando
    


In [853]:
carroX = 10
carroY = 0.0
carroZ = 0.0
carroAng = 0.0

def desenha_carro():
    global carroX, carroY, carroAng

    # aplica a matriz model
    
    # rotacao
    angle = carroAng;
    r_x = 1.0; r_y = carroAng; r_z = 0.0
    
    # translacao
    t_x = carroX; t_y = carroY; t_z = carroZ
    
    # escala
    s_x = 1; s_y = 1; s_z = 1
    
    mat_model = model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z)
    loc_model = glGetUniformLocation(program, "model")
    glUniformMatrix4fv(loc_model, 1, GL_TRUE, mat_model)

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 8)
    # desenha o modelo
    # Body 
    glDrawArrays(GL_TRIANGLES , 12095, 606) ## renderizando
    # Black
    glBindTexture(GL_TEXTURE_2D, 13)

    glDrawArrays(GL_TRIANGLES , 12701, 168) ## renderizando

    # Windows
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 11)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES , 12869, 186) ## renderizando

    # Bumpers
    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 10)
    glDrawArrays(GL_TRIANGLES , 13055, 480) ## renderizando

    # Lights
    glBindTexture(GL_TEXTURE_2D, 13)
    glDrawArrays(GL_TRIANGLES,13535,12) ## renderizando

    # Bottom
    glBindTexture(GL_TEXTURE_2D, 13)
    glDrawArrays(GL_TRIANGLES , 13547, 144) ## renderizando

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 12)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES , 13691, 5376) ## renderizando

    #define id da textura do modelo
    glBindTexture(GL_TEXTURE_2D, 9)
    # desenha o modelo
    glDrawArrays(GL_TRIANGLES , 19067, 720) ## renderizando




    # #define id da textura do modelo
    # glBindTexture(GL_TEXTURE_2D, 6)
    # # desenha o modelo
    
    # glDrawArrays(GL_TRIANGLES , 12330, 20022-12330) ## renderizando



### Eventos para modificar a posição da câmera.

* Usei as teclas A, S, D e W para movimentação no espaço tridimensional.
* Usei a posição do mouse para "direcionar" a câmera.

In [854]:
cameraPos   = glm.vec3(0.0,  0.0,  1.0);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

polygonal_mode = False
prev_key = -1

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    global carroX, carroY, carroZ, carroAng, prev_key

    cameraSpeed = 0.2
    if key == 73:
        carroAng = 0
        carroZ += 0.1
    if key == 75:
        carroAng = -180
        carroZ -= 0.1
    if key == 74:
        carroAng = 90
        carroX += 0.1
    if key == 76:
        carroAng = 270
        carroX -= 0.1

        
    if key == 87 and (action==1 or action==2): # tecla W
        cameraPos += cameraSpeed * cameraFront
    
    if key == 83 and (action==1 or action==2): # tecla S
        cameraPos -= cameraSpeed * cameraFront
    
    if key == 65 and (action==1 or action==2): # tecla A
        cameraPos -= glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 68 and (action==1 or action==2): # tecla D
        cameraPos += glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.3 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


    
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Matrizes Model, View e Projection

Teremos uma aula específica para entender o seu funcionamento.

In [855]:
def model(angle, r_x, r_y, r_z, t_x, t_y, t_z, s_x, s_y, s_z):
    angle = math.radians(angle)
    
    matrix_transform = glm.mat4(1.0) # instanciando uma matriz identidade

    
    # aplicando translacao
    matrix_transform = glm.translate(matrix_transform, glm.vec3(t_x, t_y, t_z))    
    
    # aplicando rotacao
    matrix_transform = glm.rotate(matrix_transform, angle, glm.vec3(r_x, r_y, r_z))
    
    # aplicando escala
    matrix_transform = glm.scale(matrix_transform, glm.vec3(s_x, s_y, s_z))
    
    matrix_transform = np.array(matrix_transform)
    
    return matrix_transform

def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Nesse momento, exibimos a janela.

In [856]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

### Loop principal da janela.
Enquanto a janela não for fechada, esse laço será executado. É neste espaço que trabalhamos com algumas interações com a OpenGL.

In [857]:
glEnable(GL_DEPTH_TEST) ### importante para 3D
   

rotacao_inc = 0
while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
    
    

    desenha_caixa()   
    desenha_terreno()
    desenha_casa()
    
    rotacao_inc += 0.1
    desenha_monstro(rotacao_inc)
    desenha_note()
    desenha_carro()

    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_TRUE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_TRUE, mat_projection)    
    
    

    
    glfw.swap_buffers(window)

glfw.terminate()

# Exercício

* Adicione mais 2 modelos no cenário com suas respectivas texturas. Procure em repositórios abertos/gratuitos por modelos no formato Wavefront (extensão .obj). Verifique se o conteúdo das faces do modelo é baseado em triângulos. Verifique se o modelo acompanha alguma imagem (.jpg, png, etc) com a textura. Evite modelos compostos por múltiplos objetos/texturas.

* Coloque um cubo para "encapsular" todo o seu cenário. A face inferior do cubo será seu terreno. A face superior será o céu. As faces laterais serão horizontes. Crie um único arquivo de textura (imagem png ou jpg) com todas as faces. No arquivo .obj do modelo, define as coordenadas de textura para cada triângulo.